# Evaluation Notebook

**Author**: [Sayan Nath](https://twitter.com/sayannath2350)

**Description**: In this notebook, I have evaluated all the variants of the MobileViT model using the ImageNet-1k validation set.

## Initial-setup

In [1]:
!nvidia-smi

Sun Oct  9 01:04:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Imports

In [2]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow as tf

from imutils import paths
import json
import re

## Constants

In [3]:
AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 256
IMAGE_SIZE = 256
TF_MODEL_ROOT = "/content/mobilevit_models" # You can easily download the models from TensorFlow Hub.

## Set up ImageNet-1k labels

In [4]:
with open("imagenet_class_index.json", "r") as read_file:
    imagenet_labels = json.load(read_file)

MAPPING_DICT = {}
LABEL_NAMES = {}
for label_id in list(imagenet_labels.keys()):
    MAPPING_DICT[imagenet_labels[label_id][0]] = int(label_id)
    LABEL_NAMES[int(label_id)] = imagenet_labels[label_id][1]

In [12]:
all_val_paths = list(paths.list_images("val"))
all_val_labels = [MAPPING_DICT[x.split("/")[1]] for x in all_val_paths]

all_val_paths[:5], all_val_labels[:5]

(['val/n03982430/ILSVRC2012_val_00022228.JPEG',
  'val/n03982430/ILSVRC2012_val_00037815.JPEG',
  'val/n03982430/ILSVRC2012_val_00022594.JPEG',
  'val/n03982430/ILSVRC2012_val_00044617.JPEG',
  'val/n03982430/ILSVRC2012_val_00029955.JPEG'],
 [736, 736, 736, 736, 736])

## Preprocessing utilities

In [6]:
def load_and_prepare(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (288, 288), method="bilinear")
    image = image / 255
    return image, label

In [7]:
def get_preprocessing_model(input_size=256):
    preprocessing_model = keras.Sequential()
    preprocessing_model.add(layers.CenterCrop(input_size, input_size))
    return preprocessing_model

## Prepare `tf.data.Dataset`

In [8]:
preprocessor = get_preprocessing_model()

dataset = tf.data.Dataset.from_tensor_slices((all_val_paths, all_val_labels))
dataset = dataset.map(load_and_prepare, num_parallel_calls=AUTO).batch(BATCH_SIZE)
dataset = dataset.map(lambda x, y: (preprocessor(x), y), num_parallel_calls=AUTO)
dataset = dataset.prefetch(AUTO)
dataset.element_spec

(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(None,), dtype=tf.int32, name=None))

## Fetch model paths 

In [9]:
i1k_paths = tf.io.gfile.listdir(TF_MODEL_ROOT)
print(i1k_paths)

['mobilevit_xxs_1k_256', 'mobilevit_s_1k_256', 'mobilevit_xs_1k_256']


## Run evaluation

In [10]:
def get_model(model_url):
    classification_model = tf.keras.Sequential(
        [
            layers.InputLayer((256, 256, 3)),
            hub.KerasLayer(model_url),
        ]
    )
    return classification_model


def evaluate_model(model_name):
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=f"./logs/logs_{model_name}")
    model_url = TF_MODEL_ROOT + "/" + model_name
    
    model = get_model(model_url)
    model.compile(metrics=["accuracy"])
    _, accuracy = model.evaluate(dataset, callbacks=[tb_callback])
    accuracy = round(accuracy * 100, 4)
    print(f"{model_name}: {accuracy}%.", file=open(f"{model_name.strip('/')}.txt", "w"))

In [11]:
for i1k_path in i1k_paths:
    print(f"Evaluating {i1k_path}.")
    evaluate_model(i1k_path)

Evaluating mobilevit_xxs_1k_256.
196/196 [==============================] - 37s 159ms/step - loss: 0.0000e+00 - accuracy: 0.6859
Evaluating mobilevit_s_1k_256.
196/196 [==============================] - 58s 279ms/step - loss: 0.0000e+00 - accuracy: 0.7836
Evaluating mobilevit_xs_1k_256.
196/196 [==============================] - 48s 235ms/step - loss: 0.0000e+00 - accuracy: 0.7467


## Results

|      name     | original acc@1 | keras acc@1 |
|:-------------:|:--------------:|:-----------:|
| MobileViT_XXS |      69.0      |    68.59    |
|  MobileViT_XS |      74.7      |    74.67    |
|  MobileViT_S  |      78.3      |    78.36    |